## Data

In [1]:
import torch
import torch.nn as nn
import torchtext; torchtext.disable_torchtext_deprecation_warning()
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

corpus = [
    "i like a dog",
    "books are expensive"    
]
data_size = len(corpus)

# 0: noun/pronoun - 1: verb - others - 2
labels = [[0, 1, 2, 0],
          [0, 1, 2]]

# Define the max vocabulary size and sequence length
vocab_size = 9
sequence_length = 4

In [2]:
# Define tokenizer function
tokenizer = get_tokenizer('basic_english')

# Create a function to yield list of tokens
def yield_tokens(examples):
    for text in examples:
        yield tokenizer(text)

# Create vocabulary
vocab = build_vocab_from_iterator(yield_tokens(corpus),
                                  max_tokens=vocab_size,
                                  specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])
vocab.get_stoi()

{'dog': 5,
 'books': 4,
 'like': 8,
 'i': 7,
 'are': 3,
 'a': 2,
 'expensive': 6,
 '<pad>': 1,
 '<unk>': 0}

In [3]:
# Tokenize and numericalize your samples
def vectorize(text, vocab, sequence_length, sequence_label):
    tokens = tokenizer(text)
    
    token_ids = [vocab[token] for token in tokens]    
    token_ids = token_ids + [vocab["<pad>"]] * (sequence_length - len(tokens))
    sequence_label = sequence_label + [3] * (sequence_length - len(tokens))
    
    return torch.tensor(token_ids, dtype=torch.long), torch.tensor(sequence_label, dtype=torch.long)

# Vectorize the samples
sentence_vecs = []
label_vecs = []
for sentence, labels in zip(corpus, labels):
    sentence_vec, labels_vec = vectorize(sentence, vocab, sequence_length, labels)
    sentence_vecs.append(sentence_vec)
    label_vecs.append(labels_vec)

In [4]:
for v in sentence_vecs:
    print(v)

tensor([7, 8, 2, 5])
tensor([4, 3, 6, 1])


In [5]:
for v in label_vecs:
    print(v)

tensor([0, 1, 2, 0])
tensor([0, 1, 2, 3])


## Model

In [6]:
class POS_Model(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, 4)
        self.fc = nn.Linear(4, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.fc(x)
        return x.permute(0, 2, 1)

model = POS_Model(vocab_size, 4)

# test
input = torch.tensor([[7, 8, 2, 5]], dtype=torch.long)
output = model(input)
print(output.shape)

torch.Size([1, 4, 4])


## Train with full data

In [7]:
criterion = nn.CrossEntropyLoss(ignore_index=3)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [8]:
input_data = torch.tensor( [[7, 8, 2, 5],
                            [4, 3, 6, 1]], dtype=torch.long)
label_data = torch.tensor([[0, 1, 2, 0],
                           [0, 1, 2, 3]], dtype=torch.long)

for _ in range(50):
    optimizer.zero_grad()
    outputs = model(input_data)
    loss = criterion(outputs, label_data)
    print(loss.item())
    loss.backward()
    optimizer.step()

1.4671550989151
1.4352930784225464
1.4042896032333374
1.374121069908142
1.34476637840271
1.3161869049072266
1.2883341312408447
1.2611578702926636
1.234602689743042
1.2086058855056763
1.1830999851226807
1.158016324043274
1.1332916021347046
1.1088718175888062
1.0847140550613403
1.0607860088348389
1.0370656251907349
1.0135384798049927
0.9901976585388184
0.967042088508606
0.9440754055976868
0.9213047623634338
0.8987407684326172
0.8763953447341919
0.8542819619178772
0.8324142694473267
0.8108053803443909
0.7894678711891174
0.7684130072593689
0.7476506233215332
0.7271888852119446
0.7070344090461731
0.6871920228004456
0.667665421962738
0.6484571099281311
0.6295684576034546
0.6110000610351562
0.5927523374557495
0.5748249292373657
0.5572177171707153
0.5399301052093506
0.5229617357254028
0.5063122510910034
0.48998135328292847
0.47396859526634216
0.4582741856575012
0.4428980052471161
0.42784032225608826
0.41310152411460876
0.39868226647377014


In [9]:
outputs = model(input_data)
o_softmax = torch.softmax(outputs, axis=1)

print(o_softmax[0, :, 0])
print(o_softmax[0, :, 1])
print(o_softmax[0, :, 2])
print(o_softmax[0, :, 3])

print()
print(o_softmax[1, :, 0])
print(o_softmax[1, :, 1])
print(o_softmax[1, :, 2])
print(o_softmax[1, :, 3])

tensor([0.8626, 0.0235, 0.0533, 0.0606], grad_fn=<SelectBackward0>)
tensor([0.0355, 0.8868, 0.0373, 0.0404], grad_fn=<SelectBackward0>)
tensor([0.1869, 0.0188, 0.7712, 0.0231], grad_fn=<SelectBackward0>)
tensor([0.9719, 0.0054, 0.0121, 0.0106], grad_fn=<SelectBackward0>)

tensor([0.5125, 0.0456, 0.3656, 0.0763], grad_fn=<SelectBackward0>)
tensor([0.1953, 0.3521, 0.2647, 0.1879], grad_fn=<SelectBackward0>)
tensor([0.1261, 0.1224, 0.6547, 0.0968], grad_fn=<SelectBackward0>)
tensor([0.4464, 0.2565, 0.1477, 0.1494], grad_fn=<SelectBackward0>)


In [10]:
# [[0, 1, 2, 0],
#  [0, 1, 2, *]]

o_softmax.argmax(axis=1)

tensor([[0, 1, 2, 0],
        [0, 1, 2, 0]])